# Deeplearning - Anees Ahmad - 2021/08/29

# 11 Deep learning for text

## 11.3 Two approaches for representing groups of words: Sets and sequences

### 11.3.3 Processing words as a sequence: The sequence model approach

- word order matters
- sequence models are about
  - exposed the model to raw word sequences and let it figure out such features on its own

- To implement a sequence model
  - start by representing your input samples as sequences of integer indices
  - map each integer to a vector to obtain vector sequences
  - feed these sequences of vectors into a stack of layers that could cross-correlate features from adjacent vectors

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  29.8M      0  0:00:02  0:00:02 --:--:-- 29.8M


In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


#### A FIRST PRACTICAL EXAMPLE

In [ ]:
# Listing 11.12 Preparing integer sequence datasets
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)

In [ ]:
# Listing 11.13 A sequence model built on one-hot encoded vector sequences
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

In [ ]:
# Listing 11.14 Training a first basic sequence model

## Skip this part as it is too much time consuming

callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 174s 264ms/step - loss: 0.5358 - accuracy: 0.7491 - val_loss: 0.3799 - val_accuracy: 0.8398
Epoch 2/10
625/625 [==============================] - 173s 277ms/step - loss: 0.3599 - accuracy: 0.8715 - val_loss: 0.3221 - val_accuracy: 0.8802
Epoch 3/10
625/625 [==============================] - 175s 279ms/step - loss: 0.2877 - accuracy: 0.9003 - val_loss: 0.3176 - val_accuracy: 0.8874
Epoch 4/10
625/625 [==============================] - 175s 281ms/step - loss: 0.2366 - accuracy: 0.9185 - val_loss: 0.3166 - val_accuracy: 0.8880
Epoch 5/10
625/625 [==============================] - 176s 282ms/step - loss: 0.2043 - accuracy: 0.9312 - val_loss: 0.2984 - val_accuracy: 0.8940
Epoch 6/10
625/625 [==============================] - 176s 281ms/step - loss: 0.1937 - accuracy: 0.9369 - val_loss: 0.3170 - val_accuracy: 0.8816
Epoch 7/10
625/625 [==============================] - 176s 282ms/step - loss: 0.1615 - accuracy: 0.9477 - val_loss: 0.3262 -

#### UNDERSTANDING WORD EMBEDDINGS

- one-hot encoding
  - different tokens you’re encoding are all independent from each other
- Word embedding
  - Word embeddings are vector representations of words that map human language into a structured geometric space.
  - dense representations
  - structured representations

- two ways to obtain word embeddings
  - Learn word embeddings jointly with the main task you care about (such as document classification or sentiment prediction). In this setup, you start with random word vectors and then learn word vectors in the same way you learn the weights of a neural network.
  - Load into your model word embeddings that were precomputed using a different machine learning task than the one you’re trying to solve. These are called pretrained word embeddings.

#### LEARNING WORD EMBEDDINGS WITH THE EMBEDDING LAYER

In [1]:
# Donwload data and create datasets
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

# Listing 11.12 Preparing integer sequence datasets
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  50.6M      0  0:00:01  0:00:01 --:--:-- 50.6M
Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [2]:
# Listing 11.15 Instantiating an Embedding layer
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256) 

- The Embedding layer is best understood as a dictionary that maps integer indices (which stand for specific words) to dense vectors.

In [3]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               73984     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
___________________________________________________

In [4]:
# Listing 11.16 Model that uses an Embedding layer trained from scratch
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 39s 49ms/step - loss: 0.4799 - accuracy: 0.7849 - val_loss: 0.4546 - val_accuracy: 0.8374
Epoch 2/10
625/625 [==============================] - 33s 52ms/step - loss: 0.3172 - accuracy: 0.8851 - val_loss: 0.3305 - val_accuracy: 0.8750
Epoch 3/10
625/625 [==============================] - 32s 51ms/step - loss: 0.2390 - accuracy: 0.9161 - val_loss: 0.3144 - val_accuracy: 0.8742
Epoch 4/10
625/625 [==============================] - 30s 48ms/step - loss: 0.1953 - accuracy: 0.9304 - val_loss: 0.3381 - val_accuracy: 0.8824
Epoch 5/10
625/625 [==============================] - 31s 49ms/step - loss: 0.1660 - accuracy: 0.9433 - val_loss: 0.3895 - val_accuracy: 0.8802
Epoch 6/10
625/625 [==============================] - 32s 51ms/step - loss: 0.1376 - accuracy: 0.9539 - val_loss: 0.3799 - val_accuracy: 0.8782
Epoch 7/10
625/625 [==============================] - 32s 51ms/step - loss: 0.1171 - accuracy: 0.9625 - val_loss: 0.4059 - val_accuracy:

In [5]:
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 25s 31ms/step - loss: 0.3544 - accuracy: 0.8518
Test acc: 0.852


#### UNDERSTANDING PADDING AND MASKING 

In [6]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [7]:
# Listing 11.17 Using an Embedding layer with masking enabled
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 44s 59ms/step - loss: 0.3959 - accuracy: 0.8171 - val_loss: 0.2757 - val_accuracy: 0.8814
Epoch 2/10
625/625 [==============================] - 36s 57ms/step - loss: 0.2253 - accuracy: 0.9148 - val_loss: 0.2840 - val_accuracy: 0.8878
Epoch 3/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1599 - accuracy: 0.9417 - val_loss: 0.2956 - val_accuracy: 0.8910
Epoch 4/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1207 - accuracy: 0.9564 - val_loss: 0.3288 - val_accuracy: 0.8838
Epoch 5/10
625/625 [==============================] - 35s 56ms/step - loss: 0.0856 - accuracy: 0.9696 - val_loss: 0.4231 - val_accuracy: 0.8826
Epoch 6/10
625/625 [==============================] - 34s 54ms/step - loss: 0.0647 - accuracy: 0.9775 - val_loss: 0.4813 - val_accuracy: 0.8666
Epoch 7/10
625/625 [==============================] - 37s 59ms/step - loss: 0.0492 - accuracy: 0.9840 - val_loss: 0.4351 - val_accuracy:

In [8]:
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 26s 30ms/step - loss: 0.2883 - accuracy: 0.8806
Test acc: 0.881


#### USING PRETRAINED WORD EMBEDDINGS

In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-06-19 15:31:41--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-19 15:31:41--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-19 15:31:41--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [10]:
# Listing 11.18 Parsing the GloVe word-embeddings file
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [11]:
# Listing 11.19 Preparing the GloVe word-embeddings matrix
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [12]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

In [13]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________

In [14]:
callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 41s 54ms/step - loss: 0.5778 - accuracy: 0.6973 - val_loss: 0.4817 - val_accuracy: 0.7780
Epoch 2/10
625/625 [==============================] - 33s 53ms/step - loss: 0.4692 - accuracy: 0.7816 - val_loss: 0.4396 - val_accuracy: 0.7964
Epoch 3/10
625/625 [==============================] - 31s 50ms/step - loss: 0.4181 - accuracy: 0.8112 - val_loss: 0.4464 - val_accuracy: 0.8008
Epoch 4/10
625/625 [==============================] - 34s 54ms/step - loss: 0.3839 - accuracy: 0.8322 - val_loss: 0.3441 - val_accuracy: 0.8526
Epoch 5/10
625/625 [==============================] - 34s 54ms/step - loss: 0.3566 - accuracy: 0.8476 - val_loss: 0.3389 - val_accuracy: 0.8536
Epoch 6/10
625/625 [==============================] - 33s 53ms/step - loss: 0.3365 - accuracy: 0.8604 - val_loss: 0.3186 - val_accuracy: 0.8606
Epoch 7/10
625/625 [==============================] - 33s 52ms/step - loss: 0.3203 - accuracy: 0.8663 - val_loss: 0.3060 - val_accuracy:

In [15]:
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 30s 36ms/step - loss: 0.2968 - accuracy: 0.8725
Test acc: 0.873
